In [ ]:
# python dot env
import os
from dotenv import load_dotenv
from pydantic import SecretStr


In [3]:
# load environment variables
load_dotenv()


True

In [32]:
# import environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("Missing OPENAI_API_KEY")


# Open AI Implementatino VS Langchain implemenmtation with functions

## OpenAI

In [39]:

from openai import OpenAI
# create client
client = OpenAI(api_key=OPENAI_API_KEY)


# create function
def get_response_openai(input: list, model: str = 'gpt-5-nano') -> str:
    response = client.responses.create(
        model=model,
        input=input,
        )
    return response.output_text


def get_translation_openai(input_language: str, output_language: str, input_text: str) -> str:
    messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that translates text."
    },
    {
        "role": "user",
        "content": f"Translate the following text from {input_language} to {output_language}: {input_text}."
    }
]
    response = get_response_openai(messages)
    return response


get_translation_openai("English", "Spanish", "I love djing.")

'Me encanta pinchar.\n\nIf you want to emphasize being a DJ, you could also say: Me encanta ser DJ.'

## Langchain

In [40]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-5-mini", temperature=0.0)


def get_translation_langchain(input_language, output_language, input_text):
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
        ("human", "{input_text}")
    ])

    chain = prompt_template | llm

    response = chain.invoke({
        "input_language": input_language,
        "output_language": output_language,
        "input_text": input_text
    })

    return response.content

get_translation_langchain("English", "Spanish", "I love djing.")

'Me encanta pinchar.'

## Output parser

In [64]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

from langchain.output_parsers import ResponseSchema, StructuredOutputParser

def get_parser_review(review):
    llm = ChatOpenAI(model="gpt-5-mini", temperature=0.0)

    gift_schema = ResponseSchema(
        name="gift",
        description=("Was the item purchased as a gift for someone else? "
                    "Answer True if yes, False if not or unknown.")
    )
    delivery_days_schema = ResponseSchema(
        name="delivery_days",
        description=("How many days did it take for the product to arrive? "
                    "If this information is not found, output -1.")
    )
    price_value_schema = ResponseSchema(
        name="price_value",
        description=("Extract any sentences about the value or price, and output them as a comma separated Python list.")
    )

    response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

    format_instructions = output_parser.get_format_instructions()

    prompt = ChatPromptTemplate.from_messages([
        ("system", "You extract structured fields from a product review."),
        ("human",
        "Review:\n{review}\n\n"
        "Return ONLY the data following these format rules:\n{format_instructions}")
    ])

    chain = prompt | llm | output_parser

    response = chain.invoke({
        "review": review,
        "format_instructions": format_instructions
    })

    return response


In [66]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

output = get_parser_review(customer_review)
print(output)
print(type(output))

{'gift': 'True', 'delivery_days': '2', 'price_value': '["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]'}
<class 'dict'>
